In [ ]:
import numpy as np
import pandas as pd
import os, cv2
import matplotlib.pyplot as plt

In [ ]:
src = "/kaggle/input/extraction/iris_images"
dst = "/kaggle/working/iris"

img_1 = cv2.imread(src+"/Left/032L_"+str(3)+".png")
img_2 = cv2.imread(src+"/Left/032L_"+str(2)+".png")

height = min(img_1.shape[0], img_2.shape[0])
width = min(img_1.shape[1], img_2.shape[1])
img_1 = cv2.resize(img_1, (width, height))
img_2 = cv2.resize(img_2, (width, height))

In [ ]:
gray_scale1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2GRAY)
gray_scale2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2GRAY)

In [ ]:
def extract_circles(gray_scale):
    circle_info = {'center': None, 'radius': None}
    circles = cv2.HoughCircles(gray_scale, cv2.HOUGH_GRADIENT, dp=1, minDist=100,
                           param1=50, param2=30, minRadius=260, maxRadius=280)
    if circles is not None:
        center_distance = np.sqrt((circles[0,:,0] - gray_scale1.shape[1]/2)**2 + (circles[0,:,1] - gray_scale1.shape[0]/2)**2)
        closest_circle_index = np.argmin(center_distance)
        circle = circles[0, closest_circle_index, :]

        center = (int(circle[0]), int(circle[1]))
        radius = int(circle[2])

        circle_info['center'] = center
        circle_info['radius'] = radius

        detected_image = cv2.cvtColor(gray_scale1, cv2.COLOR_GRAY2BGR)
        cv2.circle(detected_image, center, radius, (0, 255, 0), 2)
        return circle_info, center, radius
    else:
        print("No circle detected.")

In [ ]:
circle_info1,center1,radius1 = extract_circles(gray_scale1)
circle_info2,center2,radius2 = extract_circles(gray_scale2)

In [ ]:
mask1 = np.zeros_like(gray_scale1)
cv2.circle(mask1, center1, radius1, (255), thickness=-1)  # Thickness of -1 fills the circle
gray_scale1 = cv2.bitwise_and(gray_scale1, gray_scale1, mask=mask1)
mask2 = np.zeros_like(gray_scale2)
cv2.circle(mask2, center2, radius2, (255), thickness=-1)  # Thickness of -1 fills the circle
gray_scale2 = cv2.bitwise_and(gray_scale2, gray_scale2, mask=mask2)
plt.figure(figsize=(10, 8))
plt.subplot(1,2,1)
plt.imshow(gray_scale1, cmap='gray')
plt.title("Extracted Iris Region")
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(gray_scale2, cmap='gray')
plt.title("Extracted Iris Region")
plt.axis('off')
plt.show()

In [ ]:
hist_image1 = cv2.calcHist([gray_scale1], [0], None, [256], [0, 256])
hist_image2 = cv2.calcHist([gray_scale2], [0], None, [256], [0, 256])

In [ ]:
gray_scale1 = cv2.resize(gray_scale1, (500, 500))
gray_scale2 = cv2.resize(gray_scale2, (500, 500))
gray_scale1 = cv2.GaussianBlur(gray_scale1, (9, 9), 1)
gray_scale2 = cv2.GaussianBlur(gray_scale2, (9, 9), 1)
gray_scale1 = cv2.equalizeHist(gray_scale1)
gray_scale2 = cv2.equalizeHist(gray_scale2)
kernel = np.ones((5,5),np.uint8)
gray_scale1 = cv2.morphologyEx(gray_scale1, cv2.MORPH_OPEN, kernel)
gray_scale2 = cv2.morphologyEx(gray_scale2, cv2.MORPH_OPEN, kernel)

In [ ]:
def point_inside_circle(point, center, radius):
    return np.linalg.norm(np.array(point) - np.array(center)) <= radius
sift = cv2.SIFT_create(nfeatures=200, contrastThreshold=0.03, edgeThreshold=11)
keypoints1, descriptors1 = sift.detectAndCompute(gray_scale1, None)
keypoints2, descriptors2 = sift.detectAndCompute(gray_scale2, None)
filtered_keypoints1 = [kp for kp in keypoints1 if point_inside_circle(kp.pt, center1, radius1)]
filtered_keypoints2 = [kp for kp in keypoints2 if point_inside_circle(kp.pt, center2, radius2)]
image1 = cv2.drawKeypoints(gray_scale1, filtered_keypoints1, None)
image2 = cv2.drawKeypoints(gray_scale2, filtered_keypoints2, None)

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)
matches = sorted(matches, key=lambda match: match.distance)
matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[:100], None)
matching_rate = len(matches) / min(len(keypoints1), len(keypoints2)) * 100

In [ ]:
print(f"Matching rate: {matching_rate:.2f}%")
scale_percent = 50
width = int(matched_image.shape[1] * scale_percent / 100)
height = int(matched_image.shape[0] * scale_percent / 100)
dim = (width, height)
resized_image = cv2.resize(matched_image, dim, interpolation=cv2.INTER_AREA)
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.show()